In [1]:
# !unzip "/content/drive/MyDrive/TSAI/data.zip" -d "/content"
# !unzip "/content/drive/MyDrive/TSAI/new-labels.zip" -d "/content"

In [2]:
import os
os.listdir(r'/content/data/train')

['dogs', 'cats']

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset, ConcatDataset
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(1)
# defining batch size for training
batch_size = 200
device

device(type='cuda')

In [5]:
device

device(type='cuda')

In [6]:
# Keyword arguments for train_loader and test _loader to load images to GPU if the available
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Trasformations on the data
transform_train =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
        transforms.RandomAffine(degrees=40, translate=None, scale=(1, 2), shear=15, resample=False, fillcolor=0),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Trasformations on the Validation
transform_test =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

BATCH_SIZE = 128
train_data = datasets.ImageFolder(root='data/train/', transform=transform_train)
trainloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
test_data = datasets.ImageFolder(root='data/validation/', transform=transform_test)
testloader  = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1362: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1376: UserWarning: Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead
  "Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead"
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slownes

In [7]:
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.vgg16(pretrained=True)

In [8]:
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 2)
model_ft.cuda()

# Specify The Layers for updating
params_to_update = []

update_params_name = ['classifier.6.weight', 'classifier.6.bias']

for name, param in model_ft.named_parameters():
    if name in update_params_name:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

classifier.6.weight
classifier.6.bias


In [10]:
def train_model(model, criterion, optimizer, scheduler, metrics,num_epochs=25):
    since = time.time()
    train_acc, train_0,train_1 = 0,0,0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            total_0 = 0
            total_1 = 0
            correct_0 = 0
            correct_1 = 0
            # Iterate over data.
            for inputs, labels in trainloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total_0 += sum(labels.data == 0)
                total_1 += sum(labels.data == 1)
                correct_0 += torch.sum(preds[labels.data == 0] == labels.data[labels.data == 0])
                correct_1 += torch.sum(preds[labels.data == 1] == labels.data[labels.data == 1])

            epoch_loss = running_loss / (total_0+total_1)
            epoch_acc = running_corrects.double() / (total_0+total_1)
            ##class 0
            class_0 = correct_0.double()/total_0
            class_1 = correct_1.double()/total_1
            print('{} class_0 Acc: {:.4f} class_1 Acc: {:.4f}'.format(phase, class_0, class_1))
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'train':
                scheduler.step()
                train_acc, train_0,train_1 = epoch_acc, class_0, class_1

            metrics = {"Train":{"Accuracy": train_acc,
                                "Class Cats": train_0,
                                "Class Dogs": train_1},
                      "Test":{"Accuracy": epoch_acc,
                                "Class Cats": class_0,
                                "Class Dogs": class_1}           
                      }
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    return model, metrics

In [11]:
import time
import copy
import pandas as pd

metrics = {"Train":{"Accuracy": 0,
                    "Class Cats": 0,
                    "Class Dogs": 0},
           "Test":{"Accuracy": 0,
                    "Class Cats": 0,
                    "Class Dogs": 0}           
           }
model_ft,metrics = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,metrics,num_epochs=25)

Epoch 0/24
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train class_0 Acc: 0.6320 class_1 Acc: 0.6620
train Loss: 0.6131 Acc: 0.6470
val class_0 Acc: 0.9500 class_1 Acc: 0.8340
val Loss: 0.3376 Acc: 0.8920
Epoch 1/24
----------
train class_0 Acc: 0.8920 class_1 Acc: 0.8500
train Loss: 0.3155 Acc: 0.8710
val class_0 Acc: 0.9060 class_1 Acc: 0.9080
val Loss: 0.2361 Acc: 0.9070
Epoch 2/24
----------
train class_0 Acc: 0.8800 class_1 Acc: 0.9020
train Loss: 0.2434 Acc: 0.8910
val class_0 Acc: 0.9500 class_1 Acc: 0.8840
val Loss: 0.2059 Acc: 0.9170
Epoch 3/24
----------
train class_0 Acc: 0.9100 class_1 Acc: 0.9060
train Loss: 0.2160 Acc: 0.9080
val class_0 Acc: 0.9320 class_1 Acc: 0.8960
val Loss: 0.1906 Acc: 0.9140
Epoch 4/24
----------
train class_0 Acc: 0.9340 class_1 Acc: 0.9000
train Loss: 0.2073 Acc: 0.9170
val class_0 Acc: 0.9320 class_1 Acc: 0.9180
val Loss: 0.1788 Acc: 0.9250
Epoch 5/24
----------
train class_0 Acc: 0.9200 class_1 Acc: 0.9260
train Loss: 0.1942 Acc: 0.9230
val class_0 Acc: 0.9300 class_1 Acc: 0.9160
val Loss: 0.1798 Ac

In [12]:
# Specify a path
PATH = "/content/drive/MyDrive/assignment_1.pt"

# Save
torch.save(model_ft.state_dict(), PATH)

import os
# get the size of file
size = os.path.getsize(PATH)
size

537085573

In [13]:
pd.DataFrame(metrics).to_csv('/content/drive/MyDrive/metrics_1.csv')
metrics

{'Test': {'Accuracy': tensor(0.9150, device='cuda:0', dtype=torch.float64),
  'Class Cats': tensor(0.9480, device='cuda:0', dtype=torch.float64),
  'Class Dogs': tensor(0.8820, device='cuda:0', dtype=torch.float64)},
 'Train': {'Accuracy': tensor(0.9040, device='cuda:0', dtype=torch.float64),
  'Class Cats': tensor(0.8980, device='cuda:0', dtype=torch.float64),
  'Class Dogs': tensor(0.9100, device='cuda:0', dtype=torch.float64)}}